In [1]:
pip install selenium pandas beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [44]:
# Student 1
# Name: Nur Syasya Masturina binti Ibrahim
# Student ID: IS01083881

# Student 2
# Name: Irdina Nadiah Binti Mohd Rizal
# Student ID: IS01083874

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

chrome_options = Options()
chrome_options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 20)

url = "https://www.newegg.com/gigabyte-gs27qa-sa-27-qhd-180-hz-super-speed-ips-black/p/N82E16824012083"
driver.get(url)

#scroll page until review tab 
for _ in range(10):
    driver.execute_script("window.scrollBy(0, 800);")
    time.sleep(1)

#click reviews tab
review_tab = wait.until(
    EC.element_to_be_clickable((By.XPATH, "//*[contains(text(),'Reviews')]"))
)
driver.execute_script("arguments[0].click();", review_tab)

#wait until the review tab section loads 
wait.until(
    EC.presence_of_element_located(
        (By.XPATH, "//div[contains(@class,'comments-cell')]")
    )
)

review_names = []
review_dates = []
review_contents = []

#scrape page function
def scrape_current_page():
    reviews = driver.find_elements(
        By.XPATH,
        "//div[contains(@class,'comments-cell') and contains(@class,'is-active')]"
    )
    #shows only visible reviews
    print("Visible reviews detected:", len(reviews))

    for review in reviews:
        try:
            #reviewer's name
            name = review.find_element(By.CLASS_NAME, "comments-name").text
        except:
            name = "N/A"

        try:
            #date of review
            date_elements = review.find_elements(By.CLASS_NAME, "comments-text")
            date = date_elements[-1].text
        except:
            date = "N/A"

        try:
            #content of review
            content = review.find_element(By.CLASS_NAME, "comments-content").text
        except:
            content = "N/A"

        review_names.append(name)
        review_dates.append(date)
        review_contents.append(content)

#scrape review from page 1 to 5
for page in range(1, 6):

    print(f"\nScraping page {page}")

    #wait for the review page number to active
    wait.until(
        EC.presence_of_element_located(
            (By.XPATH, f"//a[text()='{page}' and contains(@class,'active')]")
        )
    )

    #call scrape page function
    scrape_current_page()

    if page < 5:
        next_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, f"//a[text()='{page+1}']"))
        )
        driver.execute_script("arguments[0].click();", next_button)

        #wait until next review page becomes active
        wait.until(
            EC.presence_of_element_located(
                (By.XPATH, f"//a[text()='{page+1}' and contains(@class,'active')]")
            )
        )

        time.sleep(2)

driver.quit()

#save to CSV
df = pd.DataFrame({
    "Reviewer Name": review_names,
    "Review Date": review_dates,
    "Review Content": review_contents
})

df.to_csv("newegg_reviews_page1_to_5.csv", index=False)

print("\nScraping completed and saved to newegg_reviews_page1_to_5.csv")


Scraping page 1
Visible reviews detected: 25

Scraping page 2
Visible reviews detected: 25

Scraping page 3
Visible reviews detected: 25

Scraping page 4
Visible reviews detected: 25

Scraping page 5
Visible reviews detected: 25

Scraping completed and saved to newegg_reviews_page1_to_5.csv
